Date: October 17, 2024

Author: Mathurin Choblet

Estimated time: 30-45 minutes

# Reading Files of the NEMO-BAMHBI model

This notebook is dedicated to some pecularities to consider when reading model output from NEMO-BAMHBI,
and directed to people with little xarray and python knowledge.

Feel free to investigate the files and play with the concepts presented/mentioned here.

## Prerequisites:
* Basic concepts of xarray (https://tutorial.xarray.dev/overview/xarray-in-45-min)
* Lists in Python (https://www.w3schools.com/python/python_lists.asp)
* Unix Wildcards (question mark ? and asterisk ?)

## Data we will work with:
* Intermediate resolution data (15km): lucia:/gpfs/scratch/acad/bsmfc/historical_simulation
* High resolution data (2.5km): lucia:/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/

## Important aspects

### Model
* Naming of Nemo-Bamhbi Files
* File Sizes of High-resolution and Intermediate model runs
* unhandy coordinate dimensions

## Python
* reading single netcdf files with xarray open dataset 
* reading multiple netcdf files at once
* using **lists** with filepaths
* how to get filepaths with the glob module and wildcards
* The Problem of the RAM: what if a file is larger than RAM? Lazy loading


In [20]:
import numpy as np
import xarray as xr
import glob
import os

## Typical file names
These filenames are produced by a software called xios, which takes the nemo output, selects the output variables the modeller defined and takes the average for a specified time period.


BS_1d_19841231_19851231_btrc_T_198505-198506.nc4

* BS: is the name given to an experiment (BS: Black Sea)
* 1d: time averaging. Here, One day. So XIOS generates daily averaged output
* First set of dates (19841231_19851231): The time for which a simulation ran. Due to the high computational effort, simulations are often run for short timeslices and then restarted with the last available output
* btrc_T: The output is usually distributed a set of files. btrc_T: benthic variables, ptrc_T: pelagic variables. grid_T: grid_U: grid_V: grid_W are variables from the physical model (NEMO), which solves the equtions on different grids (https://en.wikipedia.org/wiki/Arakawa_grids). You can check which variables are in the files with the 'ncdump -h filename' command.
* the second set of dates (198505-198506): period covered by this particular output file. Note, that sometimes data from one months can be distributed across different files (this regularly happens for leap years if the time step number of the model run is not adjusted perfectly).

**These filenames have an important property: We can sort them chronologically with standard sorting methods! This is very useful when we want to load the data at once**

# File sizes of the output

* Our model products usually come at intermediate (15km) and high resolution (2.5) km. It is is important to keep in mind, that this factor ~4, means that the high resolution data is ~16 larger. So loading this data into memory will take more time, and we actually need to have the right amount of RAM allocated to our jupyter notebook to be able to read the files without crashing the notebook!
* The ultimate file size of course also depends crucially depends on how many output variables were requested by the modeller.

Lets check the time needed to read some files (this will eventually crash depending on how much RAM your notebook has, then, use open_mfdataset instead, see below)

**side note**: in jupyter notebooks, one quick way to measure the execution time of a cell, is putting %%time at the beginning of the cell

In [21]:
%%time
ds1=xr.open_dataset('/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1980-1990/BS_1d_19841231_19851231_ptrc_T_198505-198506.nc4')

CPU times: user 27.1 ms, sys: 1.01 ms, total: 28.1 ms
Wall time: 145 ms


In [22]:
%%time
ds2=xr.open_dataset('/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_ptrc_T_198505-198505.nc')

CPU times: user 29.8 ms, sys: 3 ms, total: 32.8 ms
Wall time: 41.6 ms


Loading single files is fast, but a clear difference will arise when reading many files at once. The filesize is more of a problem.

To check filesizes, we can use the numpy nbytes method. As this result is in bytes, we can convert the output into megabytes

In [23]:
ds1.nbytes/(1024**2) #38 gigabytes for high resolution data

38515.434619903564

In [24]:
ds2.nbytes/(1024**2) #1 gigabyte

1173.3823127746582

One way to avoid actually loading the data, is to use socalled lazy loading (using a package called dask in the background). Lazy loading is default behaviour with xarray open_mfdataset, which only loads the "skelleton" of the data. The data will only load it into RAM when we actually use data (for computing, plotting...). We can enforce the loading into ram with .load().

**To increase the amount of RAM your jupyter notebook has access to, change the line #SBATCH --mem-per-cpu=2GB in the .sh file that you use to start the jupyter notebook accordingly.** You then need to restart your jupyter notebook to have this amount of RAM accesible

In [25]:
ds1=xr.open_mfdataset('/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1980-1990/BS_1d_19841231_19851231_ptrc_T_198505-198506.nc4')

If we just select certain variables, the file size will be smaller of course. Keep this in mind to use open_mfdataset and finally only load into RAM the data that you really need

In [26]:
ds1['DOX'].nbytes/(1024**2) #only one gigabyte now.

1040.4502716064453

# The coordinates of the nemo output

* When reading in the nemo data, you will notice that:
    * coordinates are nested (y,x), such that we can not use our beloved DataArray.sel(lat=...,method='nearest') method
    * the name of coordinates  vary depending on the type of grid (not always, depends on the output variables)
    * Some Latitudes and Longitudes are missing.
    * the data is not necessarily land-sea masked 

This is not handy for handling the data, so its better to do a quick conversion from the start. It takes the latitudes and longitudes from the domain file of the model run. **WARNING:** The u,v and t coordinates are not actually the same (arakawa grids), but for simple output analysis, we will just ignore that.

* To circument the problem, I wrote a function that takes as arguments the dataarray/dataset with the unhandy coordinates and the corresponding domain file, it also applies a land-sea mask.

**Note:** There is probably a more elegant way to do this, or directly tell nemo to produce consisten lat-lon names, but the convert_xy function should also do the job. I still need to checkout this python package, which might produce a smoother solution https://github.com/rcaneill/xnemogcm 

In [96]:
ds1=xr.open_mfdataset('/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1980-1990/BS_1d_19841231_19851231_ptrc_T_198505-198506.nc4')
ds1

<xarray.Dataset>
Dimensions:               (y: 258, x: 578, deptht: 59, axis_nbounds: 2,
                           time_counter: 31)
Coordinates:
    nav_lat               (y, x) float32 dask.array<chunksize=(258, 578), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(258, 578), meta=np.ndarray>
  * deptht                (deptht) float32 0.2546 0.7742 ... 2.287e+03 2.434e+03
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(31,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 1985-05-03T12:00:00 ....
Dimensions without coordinates: y, x, axis_nbounds
Data variables: (12/41)
    deptht_bounds         (deptht, axis_nbounds) float32 dask.array<chunksize=(59, 2), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(31, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(31, 2), meta=np.ndarray>
    CFL                   (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    NFL                   (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    CEM                   (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    ...                    ...
    PAR                   (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    NPP                   (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    NPPint                (time_counter, y, x) float32 dask.array<chunksize=(31, 258, 578), meta=np.ndarray>
    KBIOL                 (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    KBIOS                 (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
    pH                    (time_counter, deptht, y, x) float32 dask.array<chunksize=(31, 59, 258, 578), meta=np.ndarray>
Attributes:
    name:         out/BS_1d_19841231_19851231_ptrc_T
    description:  TOP T grid variables
    title:        TOP T grid variables
    Conventions:  CF-1.6
    timeStamp:    2023-Jul-17 20:26:25 GMT
    uuid:         bba22a26-c6dc-45e5-b910-3a9cb3535cc6

In [145]:
#always use this function on your dataset with the corresponding domain path
domain_lr='/gpfs/scratch/acad/bsmfc/river_forcings/data/domain_lr.nc' #low/intermediate resolution domain file
domain_hr='/gpfs/scratch/acad/bsmfc/river_forcings/data/domain_hr.nc' #high resolution domain file

def convert_xy(ds,domain_path):    
    """  
    convert latitudes and longitudes from nested to normal.
    The basic output format is a inconvenient for me. Adjusted to different types of Nemo grids. 
    NOTE: For simplicity we attribute the domain file latitudes and longitudes to the different grids, which is strictly speaking not correct.
    
    Input:
        ds: xarray dataset (any shape)
        
    Output:
        ds: Dataset with simple 1D coordinates for latitudes and longitudes.
    """ 
    
    domain=xr.open_dataset(domain_path)
    lon=domain['nav_lon'][0,:].values
    lat=domain['nav_lat'][:,0].values
    
    ds=ds.rename({'y':'lat','x':'lon'})
    #assignment of new coordinates
    ds['lat']=('lat',lat)
    ds['lon']=('lon',lon)
    ds=ds.drop(('nav_lat','nav_lon'))

    #only keep at least 3 dimension variables
    ds = ds.drop_vars([var for var in ds.data_vars if ds[var].ndim < 3])
    
    #land sea masking, for this, we use top_level variable (strictly speaking, the follwing only does the masking for the first layer)
    lsmask=domain['top_level'].squeeze().rename({'y':'lat','x':'lon'})
    lsmask['lat']=lat
    lsmask['lon']=lon

    ds=xr.where(lsmask==1,ds,np.nan)
    return ds

In [147]:
ds1=xr.open_mfdataset('/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1980-1990/BS_1d_19841231_19851231_ptrc_T_198505-198506.nc4')
ds1_new=convert_xy(ds1,domain_hr)
ds1_new

<xarray.Dataset>
Dimensions:        (lat: 258, lon: 578, time_counter: 31, deptht: 59)
Coordinates:
  * time_counter   (time_counter) datetime64[ns] 1985-05-03T12:00:00 ... 1985...
  * lat            (lat) float32 40.9 40.92 40.95 40.97 ... 47.28 47.3 47.33
  * lon            (lon) float32 27.4 27.42 27.45 27.48 ... 41.78 41.8 41.83
  * deptht         (deptht) float32 0.2546 0.7742 1.316 ... 2.287e+03 2.434e+03
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(31,), meta=np.ndarray>
Data variables: (12/38)
    CFL            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    NFL            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    CEM            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    NEM            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    CDI            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    NDI            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    ...             ...
    PAR            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    NPP            (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    NPPint         (lat, lon, time_counter) float32 dask.array<chunksize=(258, 578, 31), meta=np.ndarray>
    KBIOL          (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    KBIOS          (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>
    pH             (lat, lon, time_counter, deptht) float32 dask.array<chunksize=(258, 578, 31, 59), meta=np.ndarray>

In [151]:
#code to test that that function for conversion of lat lon works properly for all grid types:
for datatype in ['ptrc_T','btrc_T','grid_U','grid_T','grid_V','grid_W']:
    ds1=xr.open_mfdataset(f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1980-1990/BS_1d_19841231_19851231_{datatype}_198505-198506.nc4')
    ds1_new=convert_xy(ds1,domain_hr)
    print(ds1_new.dims,ds1_new.coords)

#same for intermed resol. 
#do not forget to replace the domain path!
#for datatype in ['ptrc_T','btrc_T','grid_U','grid_T','grid_V','grid_W']:
for datatype in ['ptrc_T','grid_U','grid_T','grid_V']: #I did not transfer btrc and grid_W to lucia)
    ds1=xr.open_mfdataset(f'/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_{datatype}_198510-198510.nc')
    ds1_new=convert_xy(ds1,domain_lr)
    print(ds1_new.dims,ds1_new.coords)

Frozen({'lat': 258, 'lon': 578, 'time_counter': 31, 'deptht': 59}) Coordinates:
  * time_counter   (time_counter) datetime64[ns] 1985-05-03T12:00:00 ... 1985...
  * lat            (lat) float32 40.9 40.92 40.95 40.97 ... 47.28 47.3 47.33
  * lon            (lon) float32 27.4 27.42 27.45 27.48 ... 41.78 41.8 41.83
  * deptht         (deptht) float32 0.2546 0.7742 1.316 ... 2.287e+03 2.434e+03
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(31,), meta=np.ndarray>
Frozen({'lat': 258, 'lon': 578, 'time_counter': 31, 'benlvl': 1}) Coordinates:
  * time_counter   (time_counter) datetime64[ns] 1985-05-03T12:00:00 ... 1985...
  * lat            (lat) float32 40.9 40.92 40.95 40.97 ... 47.28 47.3 47.33
  * lon            (lon) float32 27.4 27.42 27.45 27.48 ... 41.78 41.8 41.83
  * benlvl         (benlvl) float32 0.1
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(31,), meta=np.ndarray>
Frozen({'lat': 258, 'lon': 578, 'time_counter': 31, 'depthu': 5

# Reading multiple files at once

* To read multiple netcdf files at once
  
        * option 1: loop over files and concatenate them with xarray along time_counter dimension
        * option 2: straight forward using xarray open_mfdataset, which takes a list of filepaths. You can preselect variables and do other modifications while loading the data via the preprocess argument (https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html)
<br/><br/>
* To generate a list of filespaths, we will use the glob.glob() method with the powerful unix concept of wildcards
* Knowing how the nemo output files are named (see above) and using wildcards, we can easily subselect paths for a specific period if we don't want/can load everything at once
* **NOTE:** The output from glob glob is not sorted (fun thread: https://www.reddit.com/r/Python/comments/dh2qwd/more_than_100_scientific_articles_may_be/). We can easily sort it with the sorted method.
* Advanced: Sorting simply takes the entire path, not only the filename, if you need to sort only on the actual filename because of your folder structure, you can pass the key argument to sorted with os.path.basename() (https://www.geeksforgeeks.org/python-os-path-basename-method/) with a lambda function: sorted(paths,key=lambda p: os.path.basename(p))

Common bugs:
* sometimes model output can be broken (e.g. no data was written, and the time axis is actually empty. To circumvent this problem, we need to figure out which files are broken and leave them out, for that there is usually no easy fix, we have to loop over the files and inspect them in someway. This is the case for some of our high resolution data. I already identified the files we'll leave out

In [152]:
#reading intermediate resolution

#which data variables do you need? exchange datatype accordingly.
datatype='grid_T'
paths=glob.glob(f'/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/*{datatype}*.nc')

#this would be equivalent to, but the above f-string substitution is handier.
paths=glob.glob(f'/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/*grid_T*.nc')
paths

['/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_grid_T_198510-198510.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19900101_20091231_grid_T_200911-200911.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_grid_T_198903-198903.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_grid_T_198808-198808.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_196808-196808.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_grid_T_197504-197504.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_grid_T_197603-197603.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195601-195601.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19700101_19891231_grid_T_198601-198601.nc',
 '/gpfs/scratch/aca

In [153]:
#paths is not sorted yet!
paths_sorted=sorted(paths)
paths_sorted
#now they are!
#to sort only using the actual filename, not the entire path
#paths_sorted=sorted(paths,key=lambda p: os.path.basename(p))

['/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195001-195001.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195002-195002.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195003-195003.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195004-195004.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195005-195005.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195006-195006.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195007-195007.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195008-195008.nc',
 '/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/BS_1d_19500101_19691231_grid_T_195009-195009.nc',
 '/gpfs/scratch/aca

In [167]:
%%time
#charging everything will take about a minute, even if its with mfdataset (lazy loading) for brevity, see the subselection next
data_all=xr.open_mfdataset(paths_sorted)

CPU times: user 39.2 s, sys: 2.88 s, total: 42.1 s
Wall time: 1min 6s


In [166]:
%%time

#only interested in loading a subset of data? Lets create a list of paths only for the 1980s using the ? wildcard.
#here, the use the fact that the output comes in monthly files

datatype='grid_T'
paths80s=sorted(glob.glob(f'/gpfs/scratch/acad/bsmfc/historical_simulation/1950_2014/*{datatype}_198???-198???.nc'))
data80s=xr.open_mfdataset(paths80s) #the data is only lazy loaded here
data80s=convert_xy(data80s,domain_lr)

print(data80s.nbytes/(1024**2)) #9gigabytes for the grid_T data of the eighties

15296.798851013184
CPU times: user 3.24 s, sys: 276 ms, total: 3.52 s
Wall time: 5.86 s


In [162]:
%%time
data80s.load()
#loading takes about ~28 seconds

CPU times: user 14.4 s, sys: 7.13 s, total: 21.6 s
Wall time: 28.3 s


<xarray.Dataset>
Dimensions:        (lat: 44, lon: 81, time_counter: 3653, deptht: 59,
                    axis_nbounds: 2)
Coordinates:
  * time_counter   (time_counter) datetime64[ns] 1980-01-01T12:00:00 ... 1989...
  * lat            (lat) float32 41.0 41.14 41.28 41.42 ... 46.69 46.83 46.97
  * lon            (lon) float32 27.56 27.74 27.93 28.11 ... 42.0 42.18 42.37
  * deptht         (deptht) float32 0.2546 0.7742 1.316 ... 2.287e+03 2.434e+03
    time_centered  (time_counter) datetime64[ns] 1980-01-01T12:00:00 ... 1989...
    time_instant   (time_counter) datetime64[ns] 1980-01-02 ... 1990-01-01
Dimensions without coordinates: axis_nbounds
Data variables: (12/17)
    deptht_bounds  (lat, lon, time_counter, deptht, axis_nbounds) float32 nan...
    ssh            (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    votemper       (lat, lon, time_counter, deptht) float32 nan nan ... nan nan
    vosaline       (lat, lon, time_counter, deptht) float32 nan nan ... nan nan
    mldkz5         (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    mldr10_1       (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    ...             ...
    qns            (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    qt             (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    sfx            (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    taum           (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    windsp         (lat, lon, time_counter) float32 nan nan nan ... nan nan nan
    precip         (lat, lon, time_counter) float32 nan nan nan ... nan nan nan

# Reading Highresolution data:

* this is a bit more trickier, because some files are broken. In addition some folders contain some additional files that we don't need (bis-folders, -part2 folder, FIXED files). we will exclude them looping over the filenames and checking the names

In [169]:
#missing data I have identified
datatype='grid_T'
missing = [ 
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1999/BS_1d_19990521_19990521_{datatype}_19990521-19990521.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1999/BS_1d_19991002_19991002_{datatype}_19991002-19991002.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/1999/BS_1d_19991015_19991015_{datatype}_19991015-19991015.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2000/BS_1d_20000121_20000121_{datatype}_20000121-20000121.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2000/BS_1d_20000602_20000602_{datatype}_20000602-20000602.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2001/BS_1d_20010802_20010802_{datatype}_20010802-20010802.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2003/BS_1d_20030616_20030616_{datatype}_20030616-20030616.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2004/BS_1d_20041124_20041124_{datatype}_20041124-20041124.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2006/BS_1d_20061116_20061116_{datatype}_20061116-20061116.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2007/BS_1d_20070407_20070407_{datatype}_20070407-20070407.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2007/BS_1d_20071019_20071019_{datatype}_20071019-20071019.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2009/BS_1d_20090418_20090418_{datatype}_20090418-20090418.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2010/BS_1d_20100112_20100112_{datatype}_20100112-20100112.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2014/BS_1d_20141004_20141004_{datatype}_20141004-20141004.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2015/BS_1d_20150405_20150405_{datatype}_20150405-20150405.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2016/BS_1d_20161225_20161225_{datatype}_20161225-20161225.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2017/BS_1d_20170223_20170223_{datatype}_20170223-20170223.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2018/BS_1d_20180915_20180915_{datatype}_20180915-20180915.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2020/BS_1d_20200112_20200112_{datatype}_20200112-20200112.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2021/BS_1d_20210714_20210714_{datatype}_20210714-20210714.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2021/BS_1d_20210727_20210727_{datatype}_20210727-20210727.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2021/BS_1d_20210829_20210829_{datatype}_20210829-20210829.nc4',
    f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/2021/BS_1d_20210906_20210906_{datatype}_20210906-20210906.nc4',
]

In [170]:
#as for the low resolution data, we use glob glob, sorted and wildcards, but additionally exclude files that are not needed

datatype='grid_T'
paths = sorted(glob.glob(f'/gpfs/projects/acad/bsmfc/nemo4.2.0/BSFS_BIO/output_HR001/*/BS_1d*_{datatype}*'))
#exclude the stuff named above. the following is just a concise way to write a loop (list comprehension)
pathsfiltered = [p for p in paths if not (('part1' in p) or ('bis' in p) or ('FIXED' in p) or (p in missing))]

#list comprehension is the same as doing the following (no speed advantage, its just elegant :) )
pathsfiltered2 = []
for p in paths:
    # Start by assuming we want to include this path
    include_path = True   
    # Check each condition separately
    if 'part1' in p:
        include_path = False
    if 'bis' in p:
        include_path = False
    if 'FIXED' in p:
        include_path = False
    if p in missing:
        include_path = False
    # If we still want to include the path after all checks, add it to pathsfiltered
    if include_path:
        pathsfiltered2.append(p)

# do not forget to sort your paths
pathsfiltered2=sorted(pathsfiltered2)

print('Is the output of pathsfiltered and pathsfiltered2 is the same?', pathsfiltered2==pathsfiltered)

Is the output of pathsfiltered and pathsfiltered2 is the same? True


In [171]:
%%time
#load everthing (this may take some time, I will just load the first 12 files for convenience
data_hr=xr.open_mfdataset(pathsfiltered[:12])

CPU times: user 396 ms, sys: 39.4 ms, total: 435 ms
Wall time: 1.89 s


In [172]:
data_hr.nbytes/(1024**2) #size in MB

39560.060527801514

In [ ]:
%%time 

#load everthing takes about ..., and that hasn't even load the data into the RAM!
data_hr=xr.open_mfdataset(pathsfiltered)

# Filling the missing days in the high resolution output

* an unsophisticated way of fixing the missing days, is to use the xarray resample function to enforce a dayly resolution and forward fill the valid values to the missing values (NaNs).

* Its better to do that on the data thas has been loaded into the RAM (data_hr=data_hr.load())

In [174]:
data_hr=data_hr.resample(time_counter='D').ffill()

/gpfs/home/acad/ulg-mast/mchoblet/.conda/envs/env/lib/python3.11/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/gpfs/home/acad/ulg-mast/mchoblet/.conda/envs/env/lib/python3.11/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[in